<a href="https://colab.research.google.com/github/ds4geo/ds4geo/blob/master/DS4GEO_L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Science for Geoscientists - Winter Semester 2020**
# **Session 2**

In the previous session, we handled data in a very simple way using pandas. In this session we will introduce a few other helpful python object types for handling data, and expecially learn how to index/slice data (extract only certain parts of the data/object). Specifically, we will cover lists, dictionaries, and arrays from the numpy library.

We will also introduce simple array operations and aggregations, then apply these topics to a worked example from the geosciences.





# Section 1 - Lists, Dictionaries and Indexing

Lists and dictionaries are built-in python objects useful for storing and handling data.

# Lists

In [ ]:
# Python lists are ordered collections of other python objects, separated by commas
# They are defined by square brackets []
a = [1,2,3] # List of integers
b = [1.5, 2.5, 3.5] # List of floats

In [ ]:
# Lists can contain different types

# Section 2 - Numpy 1

Overview, creating arrays, and array operations



# Section 3 - Numpy 2

Aggregations and Indexing

# Section 4 - LA-ICPMS data reduction excercise

Motivation and overview